In [18]:
from AnthropicWrapper import ClaudeChat, ClaudeChatCV
from HumeWrapper import HumeSentimentAnalyzer
#from ConversationVerifier import ConversationVerifier
from dotenv import load_dotenv
from joblib import load
import os

In [11]:
# -- Global Variables --
chatlog = load('conversation.joblib') # Loading in the list of dicts

In [4]:
dotenv_path = os.path.join(os.path.dirname(os.getcwd()), ".env")
load_dotenv(dotenv_path)

True

In [ ]:
# Initialising the Sentiment Analyser
sentiment_analyser = HumeSentimentAnalyzer(os.getenv("HUME_API_KEY"))

In [ ]:
# Initialising Sentiment Summariser
chat_model_name = "claude-3-5-sonnet-20240620"

system_prompt = """You are a skilled emotions analyst provided with a detailed breakdown of sentiment analysis scores from Hume.ai, for a single response in an interview to a question from the interviewer. The scores are split into 3 sections. All numbers are from 0 to 1, linearly scaling, with 1 being a very strong representation of the indicator in question.

First, Emotions. This contains several human emotions with a numerical value indicating the strength of the corresponding emotion.
Second, Sentiments. This contains a scale from 1 to 9, each containing a numerical value indicating the magnitude of the sentiment of the topic of the conversation. A negative topic such as murder will have a high value lower in the scale, such as 1 or 2, and a positive topic will have a high value from 0 to 1 higher in the scale such as 8 or 9.
Third, Toxicity. This contains several toxic representations such as hate, insult, etc, with a value from 0 to 1 for each representation identified in the audio.

Your job is to provide a concise detailed one sentence breakdown of how the individual was feeling for the particular scores provided. You must be highly objective as your job is to discern whether or not a candidate was exhibiting traits which would or would not be fitting for a successful interview. 
Model your answer beginning with something along the lines of "For this particular response, the candidate...".
"""

sentiment_analyser = ClaudeChat(chat_model_name, system_prompt)

In [ ]:
# Initialising verifier
# TBD

In [ ]:
# Initialising Evaluation LLM

chat_model_name = "claude-3-5-sonnet-20240620"

system_prompt = "Claude, write me a sonnet about the beauty of the universe."

sentiment_analyser = ClaudeChatCV(chat_model_name, system_prompt)

In [ ]:
result = sentiment_analyser.analyze_audio(os.path.join(script_directory, "interview_audio.wav"))
